In [122]:
import requests
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus
from bs4 import BeautifulSoup

import pandas as pd
import csv
import time
import json
from pandas import json_normalize

### CNP_CD = 지역코드
11	서울특별시
26	부산광역시
27	대구광역시
28	인천광역시
29	광주광역시
30	대전광역시
31	울산광역시
36110	세종특별자치시
41	경기도
42	강원도
43	충청북도
44	충청남도
45	전라북도
46	전라남도
47	경상북도
48	경상남도
50	제주특별자치도



### PG_SZ	한 페이지 결과 수	

### PAGE	페이지 번호	

### SPL_TP_CD	공급유형코드		
국민임대 07
공공임대 08
영구임대 09
행복주택 10
장기전세 11
매입임대 13
전세임대 17	공급유형코드


In [123]:
local = [26, 27, 28, 29, 30, 31, 36110, 41, 42, 43, 44, 45, 46, 47, 48, 50]

api_key="vbUEsiJrUU+srA6G1WFZRY96t6LNyEHI7D4pvyQhU0L70CA8Dg9MoNGjlZuCTyVmeTnNtNZ0W9kRW2u/YpaIPw=="

url = f'http://apis.data.go.kr/B552555/lhLeaseInfo/lhLeaseInfo'

queryParams = '?' + urlencode({ quote_plus('ServiceKey') : api_key, quote_plus('PG_SZ') : '1000', quote_plus('PAGE') : '1', 
                               quote_plus('CNP_CD') : '11', quote_plus('SPL_TP_CD') : '07' })
url = url + queryParams

# 사이트 요청 및 파싱
res = requests.get(url)
root = json.loads(res.text)
json_object = root[1]
data = json_object['dsList']
df = json_normalize(data)


# 지역별 데이터프레임 합치기
for i in local:
    api_key="vbUEsiJrUU+srA6G1WFZRY96t6LNyEHI7D4pvyQhU0L70CA8Dg9MoNGjlZuCTyVmeTnNtNZ0W9kRW2u/YpaIPw=="
    url = f'http://apis.data.go.kr/B552555/lhLeaseInfo/lhLeaseInfo'
    queryParams = '?' + urlencode({ quote_plus('ServiceKey') : api_key, quote_plus('PG_SZ') : '1000', quote_plus('PAGE') : '1', 
                               quote_plus('CNP_CD') : f'{i}', quote_plus('SPL_TP_CD') : '07' })
    url = url + queryParams
    
    # 사이트 요청 및 파싱
    res = requests.get(url)
    root = json.loads(res.text)
    json_object = root[1]
    data = json_object['dsList']
    df1 = json_normalize(data)
    df = df.append(df1, ignore_index=True)

,DDO_AR,RFE,SUM_HSH_CNT,RNUM,SBD_LGO_NM,MVIN_XPC_YM,ARA_NM,HSH_CNT,ALL_CNT,AIS_TP_CD_NM,LS_GMY
0,46.71,358790,873,1,서울강남 3블록,201311,서울특별시 강남구,72,19,국민임대,43231000
1,46.05,358790,873,2,서울강남 3블록,201311,서울특별시 강남구,64,19,국민임대,43231000
2,46.01,358790,873,3,서울강남 3블록,201311,서울특별시 강남구,180,19,국민임대,43231000
3,36.62,252360,873,4,서울강남 3블록,201311,서울특별시 강남구,84,19,국민임대,20188000
4,36.08,252360,873,5,서울강남 3블록,201311,서울특별시 강남구,448,19,국민임대,20188000
...,...,...,...,...,...,...,...,...,...,...,...
2977,46.7203,81540,151,66,함덕진우,,제주특별자치도 제주시,16,70,국민임대,14833000
2978,37.2671,50350,151,67,함덕진우,,제주특별자치도 제주시,3,70,국민임대,9221000
2979,37.2638,50350,151,68,함덕진우,,제주특별자치도 제주시,3,70,국민임대,9221000
2980,37.2011,55150,151,69,함덕진우,,제주특별자치도 제주시,4,70,국민임대,10036000


In [126]:
df1 = df.rename(columns={'DDO_AR':'전용면적', 'RFE':'월임대료','SUM_HSH_CNT':'총세대수', 'RNUM':'데이터순번', 'SBD_LGO_NM':'단지명', 
                         'MVIN_XPC_YM':'최초입주년월', 'ARA_NM':'지역명', 'HSH_CNT':'세대수', 'ALL_CNT':'전체건수', 
                         'AIS_TP_CD_NM':'공급유형명', 'LS_GMY':'임대보증금'})

df1.to_csv('임대주택단지.csv', encoding='cp949')

In [ ]:
'''
컬럼명
'dsList': [{
    'DDO_AR': '전용면적',
    'RFE': '월임대료',
    'SUM_HSH_CNT': '총세대수',
    'RNUM': '데이터순번',
    'SBD_LGO_NM': '단지명',
    'MVIN_XPC_YM': '최초입주년월',
    'ARA_NM': '지역명',
    'HSH_CNT': '세대수',
    'ALL_CNT': '전체건수',
    'AIS_TP_CD_NM': '공급유형명',
    'LS_GMY': '임대보증금'
'''

In [ ]:
'''
현재 결측치 
임대보증금        569
임대료            569

도보 10분거리 내 지하철역 수(환승노선 수 반영)  211
도보 10분거리 내 버스정류장 수      4
'''

'''
총세대수, 임대건물구분, 지역, 공급유형, 전용면적, 전용면적별세대수, 공가수, 자격유형,
임대보증금, 임대료, 도보 10분거리내 지하철, 도보 10분거리 내 버스정류장
'''
'''
필요한 컬럼

전용면적, 월임대료, 총세대수, 지역명, 공급유형명, 임대보증금 
'''